In [1]:
import numpy as np
import gzip
import struct

from matplotlib import pyplot as plt

In [2]:
def get_labels():
    with gzip.open("train_labels.gz", 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.fromstring(f.read(), dtype=np.uint8).reshape(shape)
    
def get_features():
    with gzip.open("train_features.gz", 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.fromstring(f.read(), dtype=np.uint8).reshape(shape)
    
def plot_image(image):
    plt.imshow(image.reshape((28,28)), cmap="gray")
    
labels = get_labels()
features = get_features().reshape((60000, 784))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  """
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  # This is added back by InteractiveShellApp.init_path()


In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(0 - x))

def d_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

def squared_loss(y, x):
    return (y - x) ** 2

def d_squared_loss(y, x):
    return - 2 * (y - x)

In [79]:
def run(xs, w_1, w_2, b_1, b_2):
    """
    :param x: the matrix of input data, shape (N, D)
    """
    N = xs.shape[0]
    D = xs.shape[1]
    xs = xs.T
    f1 = w_1 @ xs
    f2 = (b_1 + f1.T).T
    f3 = sigmoid(f2)
    f4 = w_2 @ f3
    f5 = (b_2 + f4.T).T
    f6 = sigmoid(f5)
    f7 = squared_loss(xs, f6)
    f8 = np.sum(f7, axis=0)
    f9 = np.sum(f8, axis=0)
    f10 = f9 / N
    
    t10 = 1
    t9 = 1 / N
    t8 = np.full((N), 1 / N)
    t7 = np.full((D, N), 1 / N)
    t6 = d_squared_loss(xs, f6) * t7
    t5 = (d_sigmoid(f5)) * t6
    t4 = t5
    tb2 = np.sum(t5, axis=1)
    t3 = np.sum(w_2 @ t4, axis=0)
    print(t3.shape)
#     tw2 = np.sum(f3 @ t4.T, axis=1)
# #     print(tw2.shape)
# #     print(tb2.shape)
#     t2 = d_sigmoid(f2) @ t3
#     t1 = t2
#     tb1 = np.sum(t2, axis=0)
#     tw1 = np.sum(t1 @ xs, axis=1)
#     return tw1, tw2, tb1, tb2

In [80]:
w1 = np.random.normal(0, 1, (30, 784))
w2 = np.random.normal(0, 1, (784, 30))
b1 = np.random.normal(0, 1, (30))
b2 = np.random.normal(0, 1, (784))

run(features[0:10], w1, w2, b1, b2)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


ValueError: shapes (784,30) and (784,10) not aligned: 30 (dim 1) != 784 (dim 0)